#### SGAN (Semi-supervised generative adversarial network)

In [10]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import backend as K

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (Activation, BatchNormalization, Concatenate,
                                    Dense, Dropout, Flatten, Input, Lambda, Reshape)
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [11]:
# 모델 입력 차원

img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)  # 입력 이미지 차원

z_dim = 100  # 생성자의 입력으로 사용할 잡음 벡터의 크기

num_classes = 10  # 데이터셋에 있는 클래스 개수

In [12]:
# 훈련 데이터셋과 테스트 데이터셋
class Dataset:
    def __init__(self, num_labeled):
        self.num_labeled = num_labeled  # 훈련에 사용할 레이블된 샘플 개수
        # MNIST 데이터셋 적재
        (self.x_train, self.y_train), (self.x_test, self.y_test) = mnist.load_data()

        def preprocess_imgs(x):
            x = (x.astype(np.float32) - 127.5) / 127.5  # (0, 255) 사이 흑백 픽셀 값을 [-1, 1] 사이로 변환
            x = np.expand_dims(x, axis=3)  # 너비 * 높이 * 채널로 이미지 차원을 확장
            return x

        def preprocess_labels(y):
            return y.reshape(-1, 1)
        
        # 훈련 데이터
        self.x_train = preprocess_imgs(self.x_train)
        self.y_train = preprocess_labels(self.y_train)

        # 테스트 데이터
        self.x_test = preprocess_imgs(self.x_test)
        self.y_test = preprocess_labels(self.y_test)

    def batch_labeled(self, batch_size):
        # 레이블이 없는 이미지의 랜덤 배치 만들기
        idx = np.random.randint(self.num_labeled, self.x_train.shape[0], batch_size)
        imgs = self.x_train[idx]
        return imgs
    
    def training_set(self):
        x_train = self.x_train[range(self.num_labeled)]
        y_train = self.y_train[range(self.num_labeled)]
        return x_train, y_train
    
    def test_set(self):
        return self.x_test, self.y_test


In [13]:
num_labeled = 100  # 사용할 레이블된 샘플 개수(나머지는 레이블 없이 사용)

dataset = Dataset(num_labeled)

In [14]:
# 생성자
def build_generator(z_dim):

    model = Sequential()

    model.add(Dense(256 * 7 * 7, input_dim=z_dim))
    model.add(Reshape((7, 7, 256)))  # 완전 연결 층을 사용해 입력을 7 * 7 * 256 크기 텐서로 변경
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))  # 7 * 7 * 256에서 14 * 14 * 128 텐서로 바꾸는 전치 합성곱 층
    model.add(BatchNormalization())  # 배치 정규화
    
    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))  # 14 * 14 * 128에서 14 * 14 * 64 텐서로 바꾸는 전치 합성곱 층
    model.add(BatchNormalization())  # 배치 정규화

    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))  # 14 * 14 * 64에서 28 * 28 * 1 텐서로 바꾸는 전치 합성곱 층
    model.add(Activation('tanh'))  # tanh 활성화 함수

    return model

In [ ]:
# 판별자
def build_discriminator(img_shape):

    model = Sequential()

    # 28 * 28 * 1에서 14 * 14 * 32 텐서로 바꾸는 합성곱 층
    model.add(
        Conv2D(32,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same'))

    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    # 14 * 14 * 32에서 7 * 7 * 64 텐서로 바꾸는 합성곱 층
    model.add(
        Conv2D(64,
               kernel_size=3,)
    )
